In [19]:
import xml.etree.ElementTree as ET
import os
import re
import pandas as pd

def get_target_xml_fullPath():
  filePath = input()
  name, ext = os.path.splitext(filePath)
  if ext != '.xml':
    return ""

  if (not os.path.exists(filePath)):
      return ""

  return filePath
    
def search_target_xml(rootDirFullPath, regex_pattern):
  match_files = []
  for root, _, files in os.walk(rootDirFullPath):
    for filename in files:
       if re.match(regex_pattern, filename):
          match_files.append(os.path.join(root, filename))
  return match_files
   
def try_get_xml_element_text(xml_root, element_full_path):
   element = xml_root.find(element_full_path)
   if element is None:
      return "none"
   
   return element.text

def get_module_descritpion_dataFrame(targetXmlFiles):
  outputData = [["ModuleName", "JPN", "ENG", "FileName"]]
  for xml in targetXmlFiles:
    xml_tree = ET.parse(xml)
    xml_root = xml_tree.getroot()

    product_name = try_get_xml_element_text(xml_root,'./AdapterInfo/ProdName')
    descTextModuleJpn = try_get_xml_element_text(xml_root,'./AdapterInfo/DescTextMulLan/JPN')
    descTextModuleEng = try_get_xml_element_text(xml_root,'./AdapterInfo/DescTextMulLan/ENG')

    data = [product_name, descTextModuleJpn, descTextModuleEng, os.path.basename(xml)]
    outputData.append(data)
    # print(outputData)
  # print(outputData)
  df = pd.DataFrame(outputData)
  return df

dirFullPath = input()
targets = search_target_xml(dirFullPath, r"367_43_2(\d{2})_1_1\.xml")

df = get_module_descritpion_dataFrame(targets)

xml_tree = ET.parse(targets[2])
print(xml_tree.getroot())
params = xml_tree.findall('.//ParameterTable/Parameter')

language = ['JPN', 'ENG']
for l in language:
  for p in params:
    commnet = try_get_xml_element_text(p, f'ParameterNameMulLan/{l}')
    description = try_get_xml_element_text(p, f'HelpStringMulLan/{l}')
    values = try_get_xml_element_text(p, f'EnumMulLan/{l}')
    print(commnet, description, values)

print(l)



<Element 'EDSInfo' at 0x000001AA7C7721D0>
入力時定数 入力時定数を設定します。 0,"10ms",1,"1ms",2,"25μs",3,"300μs"
ID番号占有数 ID番号占有数を格納します。 None
プロダクトコード プロダクトコードを格納します None
リビジョン リビジョンを格納します。上位バイトがメジャーリビジョンを、下位バイトがマイナーリビジョンを格納します。 None
型式名 拡張ユニットの型式名を格納します。 None
デバイスタイプ デバイスタイプを格納します。 None
モニタデータBit0-15 None None
時定数us None None
Input time constant Set the input time constant. 0,"10ms",1,"1ms",2,"25us",3,"300us"
Occupied ID number Store the occupied number of the ID number. None
Product code Store the product code. None
Revision Store the revision. The high-order byte stores the major revision and the low-order byte stores the minor revision. None
Product name Store the model name of the expansion unit. None
Device type Store the device type. None
Monitor Data Bit0-15 None None
Time constant(us) None None
ENG


In [ ]:
def get_init_cmd_text_dataFrame(targetXmlFiles):
  outputData = [["ModleName", "Comment(JPN)", "Description(Jpn)", "Values(Jpn)"]]

  for xml in targetXmlFiles:
    xml_tree = ET.parse(xml)
    xml_root = xml_tree.getroot()

    mod_name = try_get_xml_element_text(xml_root,'./AdapterInfo/ProdName')
    comment_cmd_jpn = try_get_xml_element_text(xml_root, './ParameterTable/Parameter')